In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances
import matplotlib.pyplot as plt
from joblib import parallel_backend
from scipy.spatial.distance import pdist, squareform

# to fix - need to try scaling dataß

In [2]:
df = pd.read_csv('data-p4/Flights1_2019_1.csv')
df

,YEAR,DAY_OF_WEEK,FL_DATE,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN_CITY_NAME,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST_CITY_NAME,DEST_STATE_ABR,DEP_DELAY,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,Unnamed: 17
0,2019,6,2019-01-19,13487,1348702,31650,"Minneapolis, MN",11193,1119302,33105,"Cincinnati, OH",KY,-10.0,1832.0,-25.0,0.0,0.0,NaN
1,2019,7,2019-01-20,13487,1348702,31650,"Minneapolis, MN",11193,1119302,33105,"Cincinnati, OH",KY,-4.0,1825.0,-37.0,0.0,0.0,NaN
2,2019,1,2019-01-21,13487,1348702,31650,"Minneapolis, MN",11193,1119302,33105,"Cincinnati, OH",KY,-9.0,1845.0,-17.0,0.0,0.0,NaN
3,2019,2,2019-01-22,13487,1348702,31650,"Minneapolis, MN",11193,1119302,33105,"Cincinnati, OH",KY,-4.0,1839.0,-23.0,0.0,0.0,NaN
4,2019,3,2019-01-23,13487,1348702,31650,"Minneapolis, MN",11193,1119302,33105,"Cincinnati, OH",KY,-6.0,1850.0,-12.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583980,2019,3,2019-01-30,14057,1405702,34057,"Portland, OR",13930,1393007,30977,"Chicago, IL",IL,-9.0,1225.0,-27.0,0.0,0.0,NaN
583981,2019,3,2019-01-30,13930,1393007,30977,"Chicago, IL",14908,1490803,32575,"Santa Ana, CA",CA,35.0,2230.0,17.0,17.0,1.0,NaN
583982,2019,3,2019-01-30,14635,1463502,31714,"Fort Myers, FL",13930,1393007,30977,"Chicago, IL",IL,132.0,2035.0,113.0,113.0,1.0,NaN
583983,2019,3,2019-01-30,11618,1161802,31703,"Newark, NJ",13930,1393007,30977,"Chicago, IL",IL,32.0,841.0,37.0,37.0,1.0,NaN


In [11]:
df['scheduled_arrival'] = df['ARR_TIME'] - df['ARR_DELAY']

In [3]:
df.dropna(subset=['DEP_DELAY','ARR_DELAY'],inplace=True) # this is just for now to see how dtap runs, will impute properly later
#df = df.sample(n=50000, random_state=42).copy() # using a sample for now because whole dataframe is slow / causes crashes
#df = df.reset_index(drop=True)

df['DEP_DELAY_scaled'] = 2 * (df['DEP_DELAY'] - df['DEP_DELAY'].min()) / \
                        (df['DEP_DELAY'].max() - df['DEP_DELAY'].min()) - 1
df['ARR_DELAY_scaled'] = 2 * (df['ARR_DELAY'] - df['ARR_DELAY'].min()) / \
                        (df['ARR_DELAY'].max() - df['ARR_DELAY'].min()) - 1

In [12]:
delays_paired = df.groupby(['DEST_AIRPORT_ID', 'ORIGIN_AIRPORT_ID','scheduled_arrival'])[['DEP_DELAY_scaled','ARR_DELAY_scaled']].mean().reset_index()
delays_paired.dropna(axis=0)
delays_paired

,DEST_AIRPORT_ID,ORIGIN_AIRPORT_ID,scheduled_arrival,DEP_DELAY_scaled,ARR_DELAY_scaled
0,10135,10397,-8.0,-0.793875,-0.770168
1,10135,10397,1229.0,-0.939782,-0.900102
2,10135,10397,1269.0,-0.868669,-0.832850
3,10135,10397,1349.0,-0.767962,-0.726059
4,10135,10397,1674.0,-0.948763,-0.919907
...,...,...,...,...,...
127855,16218,14107,2047.0,-0.948732,-0.909888
127856,16218,14107,2084.0,-0.930506,-0.858387
127857,16218,14107,2087.0,-0.928151,-0.869414
127858,16218,14107,2351.0,-0.947782,-0.905978


In [4]:
vals = df[['DEP_DELAY_scaled','ARR_DELAY_scaled']].values.astype(np.float32)

# chunk_size = 25000 
# vals_dask = da.from_array(vals, chunks=(chunk_size, vals.shape[1]))

# old

In [5]:
# gigabytes = vals_dask.nbytes / (1024 ** 3)
# gigabytes

In [6]:
# distances = pairwise_distances(vals_dask, metric='euclidean')

# rows, cols = distances.shape
# for i in range(0, rows, chunk_size):
#     for j in range(0, cols, chunk_size):
#         print(f"Saving chunk: rows {i}:{i+chunk_size}, cols {j}:{j+chunk_size}")
#         chunk = distances[i:i+chunk_size, j:j+chunk_size].compute()
#         np.save(f"data-p4/pairwise_distances/distances_{i}_{j}.npy", chunk)  # Save as .npy for efficiency

In [5]:
batch_size = 25000  # Process in smaller chunks
n = vals.shape[0]
distances = np.zeros((n, n))

# for i in range(0, n, batch_size):
#     for j in range(0, n, batch_size):
#         distances[i:i+batch_size, j:j+batch_size] = pairwise_distances(
#             vals[i:i+batch_size],
#             vals[j:j+batch_size]
#         )

In [ ]:
# for i in range(0, n, batch_size):
#     for j in range(0, n, batch_size):
#         print(f"Processing block ({i}:{i+batch_size}, {j}:{j+batch_size})...")
#         distances = pairwise_distances(
#             vals[i:i+batch_size],
#             vals[j:j+batch_size],
#             n_jobs=-1
#         ).astype(np.float32)  # Use float32 to reduce memory
#         # Save each chunk
#         pd.DataFrame(distances).to_csv(f"data-p4/pairwise_distances/distances_{i}_{j}.csv", index=False)

for i in range(0, n, batch_size):
    for j in range(0, n, batch_size):
        print(f"Processing block ({i}:{i+batch_size}, {j}:{j+batch_size})...")
        
        # Use joblib's parallel backend for Scikit-learn
        with parallel_backend('threading', n_jobs=-1):
            distances = pairwise_distances(
                vals[i:i+batch_size],
                vals[j:j+batch_size],
                metric='euclidean'
            ).astype(np.float32)  # Use float32 to reduce memory usage
        
        # Save each chunk as a .npy file
        np.save(f"data-p4/pairwise_distances/distances_{i}_{j}.npy", distances)

Processing block (0:25000, 0:25000)...
Processing block (0:25000, 25000:50000)...
Processing block (0:25000, 50000:75000)...
Processing block (0:25000, 75000:100000)...
Processing block (0:25000, 100000:125000)...
Processing block (0:25000, 125000:150000)...
Processing block (0:25000, 150000:175000)...
Processing block (0:25000, 175000:200000)...
Processing block (0:25000, 200000:225000)...
Processing block (0:25000, 225000:250000)...
Processing block (0:25000, 250000:275000)...
Processing block (0:25000, 275000:300000)...
Processing block (0:25000, 300000:325000)...
Processing block (0:25000, 325000:350000)...
Processing block (0:25000, 350000:375000)...
Processing block (0:25000, 375000:400000)...
Processing block (0:25000, 400000:425000)...
Processing block (0:25000, 425000:450000)...
Processing block (0:25000, 450000:475000)...
Processing block (0:25000, 475000:500000)...
Processing block (0:25000, 500000:525000)...
Processing block (0:25000, 525000:550000)...
Processing block (0:2

In [ ]:
distances = pairwise_distances(vals, metric='sqeuclidean',n_jobs= -1)

In [ ]:
df['dtap'] = distances.sum(axis=1)

In [ ]:
plt.scatter(df.index, df['dtap'])
plt.title("Distance to All Points")
plt.xlabel("Index")
plt.ylabel("DTAP")
plt.show()

In [ ]:
df['dtap'].nlargest(10).index

In [ ]:
np.fill_diagonal(distances, np.inf)
df['dtnn'] = np.min(distances, axis=1)

In [ ]:
plt.scatter(df.index, df['dtnn'])
plt.title("Distance to Nearest Neighbour")
plt.xlabel("Index")
plt.ylabel("DTNN")
plt.show()

In [ ]:
df['dtnn'].nlargest(10).index

In [ ]:
df.iloc[[28535]]

In [ ]:
# k = 5 
# df['dtknn'] = np.sort(distances, axis=1)[:, 1:k+1].sum(axis=1)
